In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import csv
import os
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import geoopt
from datetime import datetime
import json
import numpy as np

torch.manual_seed(42)
base_path = "paws"

# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Change as needed
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# -----------------------------
# 1. Dataset Class for Binary Paraphrase Data
# -----------------------------

class BinaryParaphraseDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=128):
        self.data = self.read_file(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def read_file(self, file_path):
        data = []
        problem_rows = 0
    
        with open(file_path, 'r', encoding='utf-8') as file:
            csv_reader = csv.reader(file, delimiter='\t', quotechar=None)
            headers = next(csv_reader, None)  # Read and skip the header row
    
            for row in csv_reader:
                if len(row) == 4:
                    sentence1, sentence2, label_str = row[1], row[2], row[3] 
                    try:
                        # Ensure label is either 0 or 1 (binary)
                        label = int(float(label_str))  # Support for both integer and float formats
                        if label not in [0, 1]:
                            # Normalize any other value to binary (0 or 1)
                            # Typically, values > 0 could be considered paraphrases
                            label = 1 if label > 0 else 0
                        data.append((sentence1.strip(), sentence2.strip(), label))
                    except:
                        continue
                else:
                    problem_rows += 1
    
        print("!!!!!!total problem rows = ", problem_rows)
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence1, sentence2, label = self.data[idx]
        
        # Tokenize both sentences
        input1 = self.tokenizer(
            sentence1, 
            padding='max_length', 
            truncation=True, 
            max_length=self.max_length, 
            return_tensors="pt",
            return_token_type_ids=False
        )
        
        input2 = self.tokenizer(
            sentence2, 
            padding='max_length', 
            truncation=True, 
            max_length=self.max_length, 
            return_tensors="pt",
            return_token_type_ids=False
        )
        
        # Remove batch dimension
        input1 = {k: v.squeeze(0) for k, v in input1.items()}
        input2 = {k: v.squeeze(0) for k, v in input2.items()}
        
        return {
            'input1': input1,
            'input2': input2,
            'label': torch.tensor(label, dtype=torch.float32)
        }

# -----------------------------
# 2. Hyperbolic Model
# -----------------------------    
class HyperbolicMapper(nn.Module):
    def __init__(self, sbert_model_name='sentence-transformers/all-MiniLM-L6-v2', output_dim=32):
        super(HyperbolicMapper, self).__init__()
        # Frozen SBERT
        self.sbert = AutoModel.from_pretrained(sbert_model_name)
        for param in self.sbert.parameters():
            param.requires_grad = False
        
        sbert_hidden_dim = self.sbert.config.hidden_size
        self.curvature = nn.Parameter(torch.tensor(1.0))
        self.temperature = nn.Parameter(torch.tensor(1.0))
        
        # Projection layer
        self.projection = nn.Sequential(
            nn.Linear(sbert_hidden_dim, output_dim))
        
        print("Initialized hyperbolic model with output dimension:", output_dim)

    def poincare_project(self, x):
        """Project vectors onto the Poincaré ball"""
        x = x / self.temperature
        norm = torch.norm(x, p=2, dim=-1, keepdim=True)
        scale = (1 - 1e-5) / torch.clamp(norm * torch.sqrt(self.curvature), min=1e-5)
        return x * scale
        
    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            sbert_output = self.sbert(input_ids=input_ids, attention_mask=attention_mask)
            cls_embedding = sbert_output.last_hidden_state[:, 0]
        
        projected = self.projection(cls_embedding)
        return self.poincare_project(projected)

# -----------------------------
# 3. Hyperbolic Distance Functions
# -----------------------------

def poincare_distance(x, y, curvature=1.0, eps=1e-5):
    """Compute Poincaré distance between vectors"""
    sqrt_c = torch.sqrt(curvature + eps)
    
    # Handle different dimensions
    if x.dim() == 2 and y.dim() == 2:
        # Both are batch of vectors
        # Compute norms
        x_norm = torch.norm(x, p=2, dim=-1, keepdim=True) * sqrt_c
        y_norm = torch.norm(y, p=2, dim=-1, keepdim=True) * sqrt_c
        
        # Compute pairwise distance
        pairwise_norm = torch.norm(x - y, p=2, dim=-1, keepdim=True) * sqrt_c
        
        # Hyperbolic distance calculation
        denominator = (1 - curvature * x_norm**2) * (1 - curvature * y_norm**2)
        inside = 1 + 2 * curvature * pairwise_norm**2 / (denominator.clamp(min=eps))
        return torch.acosh(torch.clamp(inside, min=1+eps)).squeeze(-1) / (sqrt_c + eps)
    else:
        raise ValueError(f"Incompatible shapes: x {x.shape}, y {y.shape}")


def hyperbolic_contrastive_loss(dist, labels, pos_margin=0.5, neg_margin=2.0, pos_weight=1.0, neg_weight=1.0):
    # For positive pairs: penalize if distance > pos_margin
    # For negative pairs: penalize if distance < neg_margin
    pos_loss = torch.clamp(dist - pos_margin, min=0.0) ** 2
    neg_loss = torch.clamp(neg_margin - dist, min=0.0) ** 2
    loss = pos_weight * labels * pos_loss + neg_weight * (1 - labels) * neg_loss
    return loss.mean()

# -----------------------------
# 4. Training Functions
# -----------------------------

def train_one_epoch(train_loader, model, optimizer, device):
    model.train()
    total_loss = 0
    total_samples = 0
    
    # Initialize progress bar
    prog_bar = tqdm(train_loader, desc="Training", leave=False)
    
    for batch in prog_bar:
        optimizer.zero_grad()
        
        # Get embeddings for both sentences
        input1 = {k: v.to(device) for k, v in batch['input1'].items()}
        input2 = {k: v.to(device) for k, v in batch['input2'].items()}
        
        # Forward pass
        embeds1 = model(**input1)
        embeds2 = model(**input2)
        
        # Compute hyperbolic distance
        distances = poincare_distance(embeds1, embeds2, curvature=model.curvature)
        
        # Get labels
        labels = batch['label'].to(device)
        
        # Compute loss
        loss = hyperbolic_contrastive_loss(distances, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Update tracking
        batch_size = labels.size(0)
        total_loss += loss.item() * batch_size
        total_samples += batch_size
        
        # Update progress bar
        prog_bar.set_postfix({
            'loss': loss.item(),
            'curv': model.curvature.item(),
            'temp': model.temperature.item()
        })
    
    # Return average loss
    return total_loss / total_samples if total_samples > 0 else 0

def validate(loader, model, device, threshold=0.5):
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0
    
    # Use tqdm for progress tracking
    prog_bar = tqdm(loader, desc="Validation", leave=False)
    
    with torch.no_grad():
        for batch in prog_bar:
            # Get embeddings for both sentences
            input1 = {k: v.to(device) for k, v in batch['input1'].items()}
            input2 = {k: v.to(device) for k, v in batch['input2'].items()}
            
            # Forward pass
            embeds1 = model(**input1)
            embeds2 = model(**input2)
            
            # Compute hyperbolic distance
            distances = poincare_distance(embeds1, embeds2, curvature=model.curvature)
            
            # Get labels
            labels = batch['label'].to(device)
            
            # Compute loss
            loss = hyperbolic_contrastive_loss(distances, labels)
            total_loss += loss.item() * labels.size(0)
            
            # Compute binary predictions using distance threshold
            # If distance < threshold, predict paraphrase (1), else non-paraphrase (0)
            predictions = (distances < threshold).float()
            
            # Store predictions and labels for metrics
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            # Update progress bar
            prog_bar.set_postfix({'val_loss': loss.item()})
    
    # Calculate metrics
    all_preds = np.array(all_preds).astype(int)
    all_labels = np.array(all_labels).astype(int)
    
    # Calculate accuracy manually to avoid potential issues
    accuracy = np.mean(all_preds == all_labels)
    
    # For precision, recall, and F1, handle potential errors
    try:
        # Try with average='binary' first
        precision, recall, f1, _ = precision_recall_fscore_support(
            all_labels, all_preds, average='binary', zero_division=0
        )
    except ValueError:
        # Fall back to macro averaging if we get an error
        precision, recall, f1, _ = precision_recall_fscore_support(
            all_labels, all_preds, average='macro', zero_division=0
        )
    
    # Calculate average loss
    avg_loss = total_loss / len(all_labels) if len(all_labels) > 0 else 0
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'loss': avg_loss
    }

# -----------------------------
# 5. Data Loading and Training
# -----------------------------

def collate_fn(batch):
    """Collate function for DataLoader"""
    return {
        'input1': {
            k: torch.stack([item['input1'][k] for item in batch])
            for k in batch[0]['input1']
        },
        'input2': {
            k: torch.stack([item['input2'][k] for item in batch])
            for k in batch[0]['input2']
        },
        'label': torch.stack([item['label'] for item in batch])
    }

def train_model(model, train_loader, val_loader, test_loader, label, optimizer, num_epochs=20):
    """Main training function"""
    best_val_f1 = float('-inf')
    log_data = {
        'training_start': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'config': {
            'model': label,
            'num_epochs': num_epochs,
            'optimizer': str(optimizer.__class__.__name__),
            'device': str(device)
        },
        'epochs': []
    }
    
    for epoch in range(num_epochs):
        # Initialize logging
        epoch_start = datetime.now()
        
        print(f"Epoch {epoch+1}/{num_epochs}")
    
        # Train for one epoch
        train_loss = train_one_epoch(train_loader, model, optimizer, device)
        
        # Validate
        val_metrics = validate(val_loader, model, device)

        # Track epoch data
        epoch_log = {
            'epoch': epoch + 1,
            'train_loss': float(train_loss),
            'val_loss': float(val_metrics['loss']),
            'val_accuracy': float(val_metrics['accuracy']),
            'val_precision': float(val_metrics['precision']),
            'val_recall': float(val_metrics['recall']),
            'val_f1': float(val_metrics['f1']),
            'duration_seconds': (datetime.now() - epoch_start).total_seconds(),
            'is_best': False
        }
        
        # Print metrics
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_metrics['loss']:.4f}")
        print(f"  Val Accuracy: {val_metrics['accuracy']:.4f}")
        print(f"  Val F1: {val_metrics['f1']:.4f}")
    
        # Save if best model
        if val_metrics['f1'] > best_val_f1:
            best_val_f1 = val_metrics['f1']
            torch.save(model.state_dict(), f'saved_models3/{label}.pt')
            epoch_log['is_best'] = True
            print(f"  -> Best model saved (val_f1 improved to {best_val_f1:.4f})")

        log_data['epochs'].append(epoch_log)
        
        # Save logs after each epoch (in case of crash)
        os.makedirs('model_logs3', exist_ok=True)
        with open(f'model_logs3/{label}_logs.json', 'w') as f:
            json.dump(log_data, f, indent=2)
    
    # Final evaluation on test set
    print("Evaluating on test set...")
    test_metrics = validate(test_loader, model, device)
    log_data['test_metrics'] = {
        'accuracy': float(test_metrics['accuracy']),
        'precision': float(test_metrics['precision']),
        'recall': float(test_metrics['recall']),
        'f1': float(test_metrics['f1']),
        'loss': float(test_metrics['loss'])
    }
    
    # Print test metrics
    print(f"Test Results:")
    print(f"  Accuracy: {test_metrics['accuracy']:.4f}")
    print(f"  Precision: {test_metrics['precision']:.4f}")
    print(f"  Recall: {test_metrics['recall']:.4f}")
    print(f"  F1 Score: {test_metrics['f1']:.4f}")
    
    # Add final metadata
    log_data['training_end'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_data['best_val_f1'] = float(best_val_f1)
    
    # Final save
    with open(f'model_logs3/{label}_logs.json', 'w') as f:
        json.dump(log_data, f, indent=2)
    
    return log_data


/cronus_data/vraja/dysarthria/lib/python3.11/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
def main():
    # Create directories if they don't exist
    os.makedirs('saved_models3', exist_ok=True)
    os.makedirs('model_logs3', exist_ok=True)
    
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
    
    # Load datasets
    # Update these paths to your dataset files
    train_file = f'{base_path}/train.tsv'
    val_file = f'{base_path}/dev.tsv'
    test_file = f'{base_path}/test.tsv'
    
    print(f"Loading datasets from:\n  Train: {train_file}\n Dev: {val_file}\n  Test: {test_file}")
    
    try:
        train_dataset = BinaryParaphraseDataset(train_file, tokenizer)
        val_dataset = BinaryParaphraseDataset(val_file, tokenizer)
        test_dataset = BinaryParaphraseDataset(test_file, tokenizer)
    except Exception as e:
        print(f"Error loading datasets: {e}")
        print("Please verify your file paths and format.")
        return
    
    # Count label distribution
    train_labels = [data[-1] for data in train_dataset.data]
    train_label_counts = {0: train_labels.count(0), 1: train_labels.count(1)}
    print(f"Train dataset label distribution: {train_label_counts}")

    # Count label distribution
    val_labels = [data[-1] for data in val_dataset.data]
    val_label_counts = {0: val_labels.count(0), 1: val_labels.count(1)}
    print(f"Val dataset label distribution: {val_label_counts}")
    
    test_labels = [data[-1] for data in test_dataset.data]
    test_label_counts = {0: test_labels.count(0), 1: test_labels.count(1)}
    print(f"Test dataset label distribution: {test_label_counts}")
    
    if len(train_dataset) == 0 or len(test_dataset) == 0 or len(val_dataset) == 0:
        print("Error: One or more datasets are empty. Please check your data files.")
        return
    
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=32,
        shuffle=True,
        collate_fn=collate_fn
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=32,
        shuffle=False,
        collate_fn=collate_fn
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=32,
        shuffle=False,
        collate_fn=collate_fn
    )
    
    print(f"Train size: {len(train_dataset)}")
    print(f"Val size: {len(val_dataset)}")
    print(f"Test size: {len(test_dataset)}")
    
    # Train models with different dimensions
    hyp_dims = [384, 192]
    results = {}
    
    for dim in hyp_dims:
        print(f"\nTraining model with dimension {dim}")
        # Initialize model
        model = HyperbolicMapper(output_dim=dim).to(device)
        
        # Initialize optimizer
        optimizer = geoopt.optim.RiemannianAdam(
            model.parameters(),
            lr=1e-4,
            stabilize=1000  # Helps with numerical stability
        )
        
        # Train model
        try:
            model_results = train_model(
                model=model,
                train_loader=train_loader,
                val_loader=val_loader,
                test_loader=test_loader,
                label=f"paws_hyp_{dim}",
                optimizer=optimizer,
                num_epochs=20  # Adjust as needed
            )
            
            # Store results
            results[dim] = model_results
        except Exception as e:
            print(f"Error training model with dimension {dim}: {e}")
            import traceback
            traceback.print_exc()
        
        # Clear memory
        del model
        torch.cuda.empty_cache()
    
    # Print summary of results
    print("\nResults Summary:")
    for dim, result in results.items():
        test_f1 = result['test_metrics']['f1']
        test_acc = result['test_metrics']['accuracy']
        print(f"Dimension {dim}: F1={test_f1:.4f}, Accuracy={test_acc:.4f}")


main()

Loading datasets from:
  Train: paws/train.tsv
 Dev: paws/dev.tsv
  Test: paws/test.tsv
!!!!!!total problem rows =  0
!!!!!!total problem rows =  0
!!!!!!total problem rows =  0
Train dataset label distribution: {0: 27572, 1: 21829}
Val dataset label distribution: {0: 4461, 1: 3539}
Test dataset label distribution: {0: 4464, 1: 3536}
Train size: 49401
Val size: 8000
Test size: 8000

Training model with dimension 384
Initialized hyperbolic model with output dimension: 384
Epoch 1/20


  Train Loss: 7.9516
  Val Loss: 1.3423
  Val Accuracy: 0.5440
  Val F1: 0.6026
  -> Best model saved (val_f1 improved to 0.6026)
Epoch 2/20


  Train Loss: 0.5977
  Val Loss: 1.5389
  Val Accuracy: 0.4944
  Val F1: 0.6066
  -> Best model saved (val_f1 improved to 0.6066)
Epoch 3/20


  Train Loss: 0.5764
  Val Loss: 1.5179
  Val Accuracy: 0.4999
  Val F1: 0.6074
  -> Best model saved (val_f1 improved to 0.6074)
Epoch 4/20


  Train Loss: 0.5748
  Val Loss: 1.4980
  Val Accuracy: 0.5058
  Val F1: 0.6091
  -> Best model saved (val_f1 improved to 0.6091)
Epoch 5/20


...

  Train Loss: 0.5658
  Val Loss: 1.4297
  Val Accuracy: 0.5276
  Val F1: 0.6137
  -> Best model saved (val_f1 improved to 0.6137)
Epoch 18/20


  Train Loss: 0.5662
  Val Loss: 1.4267
  Val Accuracy: 0.5279
  Val F1: 0.6131
Epoch 19/20


  Train Loss: 0.5652
  Val Loss: 1.4292
  Val Accuracy: 0.5281
  Val F1: 0.6137
  -> Best model saved (val_f1 improved to 0.6137)
Epoch 20/20


  Train Loss: 0.5651
  Val Loss: 1.4137
  Val Accuracy: 0.5331
  Val F1: 0.6142
  -> Best model saved (val_f1 improved to 0.6142)
Evaluating on test set...


Test Results:
  Accuracy: 0.5386
  Precision: 0.4873
  Recall: 0.8385
  F1 Score: 0.6164

Results Summary:
Dimension 384: F1=0.6153, Accuracy=0.5354
Dimension 192: F1=0.6164, Accuracy=0.5386
